In [143]:
import numpy as np
from sklearn.linear_model import Ridge

# Sample Dataset and Transformation

In [13]:
# dataset
Phi = np.array([[0, 2], [1, 0], [0, -2], [-1, 0], [0, 0]])
y = np.array([1, 1, 1, 1, -1])
n = 5

def transform(X):
    bias = np.repeat(1, len(X))
    dim1 = X[:, 0]**2
    dim2 = X[:, 1]**2
    dim3 = np.sqrt(2) * X[:, 0]
    dim4 = np.sqrt(2) * X[:, 1]
    dim5 = np.sqrt(2) * X[:, 0] * X[:, 1]
    return np.column_stack([bias, dim1, dim2, dim3, dim4, dim5])

new_Phi = transform(Phi)

# Constants and Helper Methods

In [149]:
# define constants
lambda_ = 2 # regularization parameter
alpha = 1e-4 # learning rate
max_iter = 1000 # maximum number of iterations
tol = 1e-4 # tolerance

In [144]:
def gradient_descent(X, y, w, gradient, alpha, max_iter, tol):
    """
    A method to perform gradient descent.

    Inputs:
    - X: A numpy array of shape (N, D) containing a minibatch of N
        data points; each point has dimension D.
    - y: A numpy array of shape (N,) containing labels for the minibatch.
    - w: A numpy array of shape (D,) containing weights.
    - gradient: A function that computes the gradient of the loss function
        with respect to weights w; the function should take the same
    - alpha: (float) learning rate for optimization.
    - max_iter: (integer) maximum number of iterations to perform.
    - tol: (float) tolerance value for stopping.

    Returns:
    - w: The learned weight vector.
    """
    for _ in range(int(max_iter)):  # set maximum number of iterations
        # update weights
        w -= alpha * gradient(X, y, w, lambda_)
        if np.linalg.norm(gradient(X, y, w, lambda_)) < tol:
            # if the norm of the gradient is smaller than the tolerance, perform early stopping
            break
    return w

def ridge_gradient_SSE(X, y, w, lambda_):
    """
    A method to compute the gradient of the SSE Ridge (l2) loss function with respect to weights w.

    Inputs:
    - X: A numpy array of shape (N, D) containing a minibatch of N
        data points; each point has dimension D.
    - y: A numpy array of shape (N,) containing labels for the minibatch.
    - w: A numpy array of shape (D,) containing weights.
    - lambda_: (float) regularization parameter.

    Returns:
    - grad: A numpy array of shape (D,) containing the gradient of the loss function with respect to w.
    """
    # compute gradient
    grad = 2 * lambda_ * w - 2 * X.T @ (y - X @ w) 
    return grad

def ridge_gradient_MSE(X, y, w, lambda_):
    """
    A method to compute the gradient of the MSE Ridge (l2) loss function with respect to weights w.

    Inputs:
    - X: A numpy array of shape (N, D) containing a minibatch of N
        data points; each point has dimension D.
    - y: A numpy array of shape (N,) containing labels for the minibatch.
    - w: A numpy array of shape (D,) containing weights.
    - lambda_: (float) regularization parameter.

    Returns:
    - grad: A numpy array of shape (D,) containing the gradient of the loss function with respect to w.
    """
    # compute gradient
    grad = 2 * lambda_ * w - 2 * X.T @ (y - X @ w) / len(X)
    return grad

In [132]:
def gradient_descent_withlog(X, y, w, gradient, alpha, max_iter, tol):
    """
    A method to perform gradient descent.

    Inputs:
    - X: A numpy array of shape (N, D) containing a minibatch of N
        data points; each point has dimension D.
    - y: A numpy array of shape (N,) containing labels for the minibatch.
    - w: A numpy array of shape (D,) containing weights.
    - gradient: A function that computes the gradient of the loss function
        with respect to weights w; the function should take the same
    - alpha: (float) learning rate for optimization.
    - max_iter: (integer) maximum number of iterations to perform.
    - tol: (float) tolerance value for stopping.

    Returns:
    - w: The learned weight vector.
    - log: A list of weights at each iteration.
    """
    log = []
    for i in range(int(max_iter)):  # set maximum number of iterations
        # update weights
        w -= alpha * gradient(X, y, w, lambda_)
        if np.linalg.norm(gradient(X, y, w, lambda_)) < tol:
            # if the norm of the gradient is smaller than the tolerance, perform early stopping
            break
        log.append(w)
    return w, log

# Perform the Experiment
The only reason is that sklearn use SSE rather than MSE as the loss function. In every gradient, the MSE is divided by the size of the dataset, while the SSE is not.
This is normally not a problem, but when we have regularization, the SSE method strength the regularization effect, while the MSE method weaken it (since we divide the loss part).
- In other words, $\lambda = 2$ in MSE is stronger in regularization than $\lambda = 2$ in SSE.

## Gradient Descent SSE vs sklearn

In [152]:
# GD with SSE
w = np.zeros(new_Phi.shape[1])
w = gradient_descent(new_Phi, y, w, ridge_gradient_SSE, 0.001, max_iter, tol)
w.round(12)

array([ 0.03042573,  0.48335373,  0.22810159,  0.        , -0.        ,
        0.        ])

In [153]:
# sklearn solution
model = Ridge(fit_intercept=False, alpha=lambda_)
model.fit(new_Phi, y)
model.coef_.round(12)

array([0.02857143, 0.48571429, 0.22857143, 0.        , 0.        ,
       0.        ])

## Gradient Descent MSE vs Close Form Solution

In [154]:
# GD with MSE
w = np.zeros(new_Phi.shape[1])
w = gradient_descent(new_Phi, y, w, ridge_gradient_MSE, 0.001, max_iter, tol)
w.round(12)

array([0.08752787, 0.15083731, 0.17377868, 0.        , 0.        ,
       0.        ])

In [155]:
# close form solution
shape = new_Phi.T.shape[0]
w_close_form = np.linalg.solve( new_Phi.T @ new_Phi + n*lambda_*np.identity(shape), new_Phi.T @ y )
w_close_form

array([0.08695652, 0.15217391, 0.17391304, 0.        , 0.        ,
       0.        ])

#### How precise can we get from gradient descent?

In [157]:
# GD with MSE
w = np.zeros(new_Phi.shape[1])
w = gradient_descent(new_Phi, y, w, ridge_gradient_MSE, alpha=1e-4, max_iter=1e5, tol=1e-6)
w.round(12)
# this is pretty close!

array([ 0.08695667,  0.15217373,  0.17391301, -0.        ,  0.        ,
        0.        ])

#### If we understand the reason, can we use ridge to obtain the same result?

In [158]:
# sklearn solution
model = Ridge(fit_intercept=False, alpha=lambda_*len(y)) # greater the regularization!
model.fit(new_Phi, y)
model.coef_.round(12)

array([0.08695652, 0.15217391, 0.17391304, 0.        , 0.        ,
       0.        ])